In [11]:
import pandas as pd
from pathlib import Path
import natsort
from skimage.io import imsave
from FLIM_functions import analyze_single_fitted_channel,generate_small_report_fig,analyze_single_fastflim_channel

In [12]:
selected_analysis_function = analyze_single_fitted_channel

In [13]:
global_path=Path(r"G:\SP8_FLIM\Treatments\Aldometanib\2024-02-02\wt_Aldo_0")

In [28]:
def in_folder_loop(global_path,selected_analysis_function):
    import pandas as pd
    from pathlib import Path
    import natsort
    from FLIM_functions import analyze_single_fitted_channel,generate_small_report_fig,analyze_single_fastflim_channel

    result_per_folder = pd.DataFrame([])
    report_per_folder = pd.DataFrame([])

    # Here we build a paired_path nested list
    paired_path_list = []
    paired_paths = []

    # Create paired paths
    for file_path in natsort.natsorted(global_path.iterdir()):
        if file_path.suffix == '.tif': 
            paired_paths.append(file_path) 
            if len(paired_paths) == 2:
                paired_path_list.append(paired_paths)
                paired_paths = []

    # Process each pair of files in the current folder
    for paired_path in paired_path_list:
        result_per_image, report_per_image, multichannel_image = selected_analysis_function(paired_path[1], paired_path[0])

        # Add identification to the result table (image level)
        result_per_image['File Name'] = paired_path[1].stem
        report_per_image['File Name'] = paired_path[1].stem


        # Concatenate the filtered results to the local table
        result_per_folder = pd.concat([result_per_folder, result_per_image])
        report_per_folder = pd.concat([report_per_folder, report_per_image]) 
        
    #start creating report images 
                # Construct the desired identification string for images 
                identification = f"{date}_{paired_path[1].parent.name}_{paired_path[1].stem}"

                #construct report images and save them 
                #small_report_fig = generate_small_report_fig(multichannel_image,identification,output_folder_path)

                #output images. only 1 chanel from param image (last chanel) 

                #image_name='param_map_' +paired_path[1].stem+'.tif'
                #imsave(outputimage_folder_path / image_name, multichannel_image[...,-1])


    return result_per_folder, report_per_folder


In [29]:
result_per_folder,report_per_folder=in_folder_loop(global_path,selected_analysis_function)
result_per_folder.to_csv("result_per_folder.csv")
report_per_folder.to_csv("report_per_folder.csv")

In [ ]:
# Create the folder where all the outputs will be collected 
output_folder_path = global_path / 'Outputs'
output_folder_path.mkdir(exist_ok=True)

outputimage_folder_path = global_path / 'Output_images'
outputimage_folder_path.mkdir(exist_ok=True)


print("Saving master table...")
# Save the master DataFrame as a CSV file
result_per_folder.to_csv(output_folder_path / "result_per_folder.csv")

print("Saving table of reports...")
# Save the full table of stat reports per each image in the tree folders
report_per_folder.to_csv(output_folder_path / "report_per_folder.csv")

print("Processing complete!")

In [25]:
import pandas as pd
import natsort
from pathlib import Path
from FLIM_functions import analyze_single_fitted_channel, generate_small_report_fig, analyze_single_fastflim_channel
from skimage.io import imsave

def in_folder_loop(global_path, selected_analysis_function, save_images=False, output_folder_path=None, date=None):
    result_per_folder = pd.DataFrame([])
    report_per_folder = pd.DataFrame([])

    # Here we build a paired_path nested list
    paired_path_list = []
    paired_paths = []

    # Create paired paths
    for file_path in natsort.natsorted(global_path.iterdir()):
        if file_path.suffix == '.tif': 
            paired_paths.append(file_path) 
            if len(paired_paths) == 2:
                paired_path_list.append(paired_paths)
                paired_paths = []

    # Process each pair of files in the current folder
    for paired_path in paired_path_list:
        result_per_image, report_per_image, multichannel_image = selected_analysis_function(paired_path[1], paired_path[0])

        # Add identification to the result table (image level)
        result_per_image['File Name'] = paired_path[1].stem
        report_per_image['File Name'] = paired_path[1].stem

        # Concatenate the filtered results to the local table
        result_per_folder = pd.concat([result_per_folder, result_per_image])
        report_per_folder = pd.concat([report_per_folder, report_per_image]) 

        # Start creating report images if save_images is True
        if save_images:
            if output_folder_path is None or date is None:
                raise ValueError("output_folder_path and date must be provided if save_images is True")

            # Construct the desired identification string for images
            identification = f"{date}_{paired_path[1].parent.name}_{paired_path[1].stem}"

            # Construct report images and save them
            small_report_fig = generate_small_report_fig(multichannel_image, identification, output_folder_path)

            # Save output images. Only 1 channel from param image (last channel)
            image_name = 'param_map_' + paired_path[1].stem + '.tif'
            imsave(output_folder_path / image_name, multichannel_image[..., -1])

    return result_per_folder, report_per_folder


In [ ]:
# Example of calling the function with saving images
result, report = in_folder_loop(global_path, selected_analysis_function, save_images=True, output_folder_path=Path("/path/to/output"), date="2023-06-17")

# Example of calling the function without saving images
result, report = in_folder_loop(global_path, selected_analysis_function)
